## Analysis sparks multiple images

In [ ]:
from os import listdir
from os.path import isfile, join
from sparks_image import *
from sparks_analysis import *
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

x_calibracion = 4.5
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Imagenes_confocal/Rata/C071112/'
files = [".".join(f.split(".")[:-1]) for f in listdir(path) if isfile(join(path, f))]
for photo_name in files:
    image = cv2.imread(path + photo_name + ".tif")    # Read image
    plt.imshow(image, cmap='gray')
    cont = input('Continue?')
    while 's' in cont.lower():
        list_img = image_process (image,path)
        list_img_col = list_img [0]
        list_img_row = list_img [1]
        out_sparks = analysis_process (list_img_col, list_img_row)
        out_sparks
        if 'n' in cont.lower():
            continue

## Sparks analysis single image

In [2]:
from sparks_image import *
from sparks_analysis import *
import cv2

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imágenes confocal/Rata/C071112/'
photo_name = 'c1ac000'
image = cv2.imread(path + photo_name + ".tif")    # Read image
list_img = image_process (image,path)
list_img_col = list_img [0]
list_img_row = list_img [1]
out_sparks = analysis_process (list_img_col, list_img_row)

LinAlgError: Incompatible dimensions

In [4]:
img_mean = (np.mean(image))
img_max = (np.max(image))
img_min = (np.min(image))
img_std = (cv2.meanStdDev(image))
print(img_mean,img_max, img_min, img_std)

31.740985870361328 177 2 (array([[31.74098587],
       [31.74098587],
       [31.74098587]]), array([[19.54016673],
       [19.54016673],
       [19.54016673]]))


In [ ]:
out_sparks.describe()

In [ ]:
out_sparks.to_csv(path+photo_name+'_sparks.csv', sep='\t')

from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

img_bc = interactive(f, brightness=100, contrast=100)   #display brightness and contrast
display (img_bc)

im = rotation (imCrop, 90)    # rotate image
display_image ('Image' , im)    # Display cropped image